In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1" 

In [ ]:
import pandas as pd 
from torch.nn.utils.rnn import pad_sequence
from transformers import Trainer,Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import AutoModelForCTC
from tqdm import tqdm
from pydub import AudioSegment
from IPython.display import Audio  # Import the audio player
import numpy as np
from jiwer import wer
from datasets import Dataset
import torch


In [ ]:
cv_valid_dev_directory = '/Users/allen/Documents/Wav2vec_Project/asr/common_voice/cv-valid-dev/cv-valid-dev'
cv_valid_dev_csv = '/Users/allen/Documents/Wav2vec_Project/asr/common_voice/cv-valid-dev.csv'
cv_valid_dev_df = pd.read_csv(cv_valid_dev_csv)
cv_valid_dev_df = cv_valid_dev_df.sample(frac=0.01, random_state=42)
ground_truth = {row['filename']: row['text'] for _, row in cv_valid_dev_df.iterrows()}
mp3_files = [f for f in os.listdir(cv_valid_dev_directory) if f.endswith('.mp3')]
wer_scores_finetuned = []
wer_scores_original = []
finetuned_model = AutoModelForCTC.from_pretrained("./wav2vec2-large-960h-cv-backup")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
# print (cv_valid_test_df)
# print (ground_truth)

hot_words = ["be careful", "destroy", "stranger"]
detected_files = []

for filepath in tqdm(cv_valid_dev_df['filename'], desc="Processing MP3 files"):
    # print (filepath)
    # Load the MP3 file using pydub
    mp3_path = '/Users/allen/Documents/Wav2vec_Project/asr/common_voice/cv-valid-dev/'+filepath
    # print (mp3_path)
    audio = AudioSegment.from_mp3(mp3_path)
    
    audio = audio.set_frame_rate(16000).set_channels(1)

    # print("Audio converted")
    
    # Load audio data from file into a format compatible with datasets
    audio_data = np.array(audio.get_array_of_samples(), dtype=np.float32)  # Ensure float32 type

    # Create a dataset from the audio data
    ds = Dataset.from_dict({
        "audio": [{"array": audio_data, "sampling_rate": 16000}]
    })

    # Tokenize input
    input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate=16000).input_values  # Batch size 1
    
    # Make the prediction
    with torch.no_grad():
        logits_finetuned = finetuned_model(input_values).logits
    
    # Decode the logits to text
    predicted_ids_finetuned = torch.argmax(logits_finetuned, dim=-1)
    transcription_finetuned = processor.decode(predicted_ids_finetuned[0])
    transcription_finetuned = transcription_finetuned.lower()
    if any(word in transcription_finetuned for word in hot_words):
        detected_files.append(filepath)
        print (transcription_finetuned)

    # print (transcription)
    
    # # Retrieve the ground truth transcription
    # ground_truth_text = ground_truth.get(filepath, None)
    # # print (ground_truth_text)
    
    # # Calculate WER if ground truth is available
    # if ground_truth_text:
    #     score_finetuned = wer(str(ground_truth_text).lower(), str(transcription_finetuned).lower())
    #     print ("Finetuned score:", score_finetuned)
    #     wer_scores_finetuned.append(score_finetuned)

# Calculate the average WER
# average_wer_finetuned = sum(wer_scores_finetuned) / len(wer_scores_finetuned) if wer_scores_finetuned else 0

# # Print the average WER
# print(f"Average WER: {average_wer:.4f}")